In [1]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 62.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.2
    Uninstalling transformers-4.56.2:
      Successfully uninstalled transformers-4.56.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 14.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu1

In [ ]:
import pandas as pd
from datasets import load_dataset

In [3]:
data = load_dataset("wikiann","en")
data

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

en/validation-00000-of-00001.parquet:   0%|          | 0.00/748k [00:00<?, ?B/s]

en/test-00000-of-00001.parquet:   0%|          | 0.00/748k [00:00<?, ?B/s]

en/train-00000-of-00001.parquet:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 20000
    })
})

In [ ]:
from datasets import load_dataset
dataset = load_dataset("wikiann", "en")
print(dataset)
print(f"Train samples: {len(dataset['train'])}")

In [ ]:
data['train'].features

In [ ]:
pd.DataFrame(data['train'][:])[['tokens','ner_tags']].iloc[0]

In [7]:
tags = data['train'].features['ner_tags']


indextotag = {idx:tag for idx,tag in enumerate(tags.feature.names)}
tagtoindex = {tag:idx for idx,tag in enumerate(tags.feature.names)}

In [8]:

tags.feature.names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [9]:
def create_tag_names(batch):
  tag_name = {'ner_tags_str' : [tags.int2str(idx) for idx in batch ['ner_tags']]}
  return tag_name

In [10]:
def create_tag_names(batch):
    ner_feature = data['train'].features['ner_tags'].feature

    tag_name = {
        'ner_tags_str': [
            [ner_feature.int2str(idx) for idx in sequence]
            for sequence in batch['ner_tags']
        ]
    }
    return tag_name
data['train'] = data['train'].map(create_tag_names, batched=True)
data['validation'] = data['validation'].map(create_tag_names, batched=True)
data['test'] = data['test'].map(create_tag_names, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [11]:
data

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans', 'ner_tags_str'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans', 'ner_tags_str'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans', 'ner_tags_str'],
        num_rows: 20000
    })
})

In [12]:
pd.DataFrame(data['train'][:])[['tokens','ner_tags','ner_tags_str']].iloc[0]

,0
tokens,"[R.H., Saunders, (, St., Lawrence, River, ), (..."
ner_tags,"[3, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0]"
ner_tags_str,"[B-ORG, I-ORG, O, B-ORG, I-ORG, I-ORG, O, O, O..."


** MODEL BUILDING**

In [13]:
from transformers import AutoTokenizer
model_checkpoint = "distilbert/distilbert-base-cased"

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [15]:
tokenizer.is_fast

True

In [16]:
inputs = data['train'][0]['tokens']
inputs = tokenizer(inputs,is_split_into_words=True)
inputs.tokens()



['[CLS]',
 'R',
 '.',
 'H',
 '.',
 'Saunders',
 '(',
 'St',
 '.',
 'Lawrence',
 'River',
 ')',
 '(',
 '96',
 '##8',
 'MW',
 ')',
 '[SEP]']

In [17]:
inputs.word_ids()

[None, 0, 0, 0, 0, 1, 2, 3, 3, 4, 5, 6, 7, 8, 8, 9, 10, None]

**Alligning with original tokens**

In [18]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)

        elif word_id is None:
            new_labels.append(-100)

        else:
            label = labels[word_id]

            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [19]:
labels = data['train'][0]['ner_tags']
word_ids = inputs.word_ids()
print(labels,word_ids)


[3, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0] [None, 0, 0, 0, 0, 1, 2, 3, 3, 4, 5, 6, 7, 8, 8, 9, 10, None]


In [20]:
align_labels_with_tokens(labels,word_ids)

[-100, 3, 4, 4, 4, 4, 0, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, -100]

In [21]:
def tokenize_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['tokens'],
        truncation=True,
        is_split_into_words=True
    )

    all_labels = examples['ner_tags']


    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs['labels'] = new_labels

    return tokenized_inputs

In [22]:
tokenized_datasets = data.map(tokenize_align_labels,batched = True,remove_columns = data['train'].column_names)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [23]:
tokenized_datasets

DatasetDict({
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 20000
    })
})

**Data collation and metrices **

In [24]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer = tokenizer)

In [25]:
batch = data_collator([tokenized_datasets['train'][i] for i in range(2)])
batch

{'input_ids': tensor([[  101,   155,   119,   145,   119, 16029,   113,  1457,   119,  4898,
          1595,   114,   113,  5306,  1604, 13277,   114,   102],
        [  101,   132,   112,   112,   112, 16357, 12221,  3680, 26305,   112,
           112,   112,   102,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]]), 'labels': tensor([[-100,    3,    4,    4,    4,    4,    0,    3,    4,    4,    4,    0,
            0,    0,    0,    0,    0, -100],
        [-100,    0,    0,    0,    0,    1,    2,    2,    2,    0,    0,    0,
         -100, -100, -100, -100, -100, -100]])}

In [26]:
!pip install seqeval
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=de1a918983d0e5796c9fe6711f99f5f5beb518332de5059ac09dffc10f676df3
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00


In [27]:
import evaluate
metric = evaluate.load('seqeval')

In [28]:
ner_feature = data['train'].features['ner_tags']
ner_feature

List(ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']))

In [29]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [30]:
labels = data['train'][0]['ner_tags']
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'I-ORG', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O']

In [31]:
predictions = labels.copy()
predictions[2] = "O"

metric.compute(predictions = [predictions],references = [labels])

{'ORG': {'precision': np.float64(1.0),
  'recall': np.float64(1.0),
  'f1': np.float64(1.0),
  'number': np.int64(2)},
 'overall_precision': np.float64(1.0),
 'overall_recall': np.float64(1.0),
 'overall_f1': np.float64(1.0),
 'overall_accuracy': 1.0}

In [ ]:
import numpy as np
def compute_metrics(eval_preds):
  logits,labels = eval_preds

  prediction = np.argmax(logits ,axis = -1)
  true_labels = [[label_names[l] for l in label if l!=-100] for label in labels]

  true_predictions = [[label_names[p] for (p, l) in zip(prediction[i], label) if l != -100]
                      for i, label in enumerate(labels)]

  all_metrices = metric.compute(predictions=true_predictions , references = true_labels)


  return {"precision": all_metrices['overall_precision'],
          "recall":all_metrices['overall_recall'],
          "f1":all_metrices['overall_f1'],
          "accuracy":all_metrices['overall_accuracy']}

In [51]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if predictions is None:
        print("ERROR: predictions is None")
        return {}
    if labels is None:
        print("ERROR: labels is None")
        return {}

    predictions = np.argmax(predictions, axis=2)

    true_predictions = []
    true_labels = []

    for prediction, label in zip(predictions, labels):
        if prediction is None or label is None:
            continue

        true_pred = []
        true_label = []

        for p, l in zip(prediction, label):
            if l != -100:
                if 0 <= p < len(label_names):
                    true_pred.append(label_names[p])
                else:
                    true_pred.append(label_names[0])

                if 0 <= l < len(label_names):
                    true_label.append(label_names[l])
                else:
                    true_label.append(label_names[0])

        if true_pred:
            true_predictions.append(true_pred)
            true_labels.append(true_label)
    if not true_predictions or not true_labels:
        return {
            "precision": 0.0,
            "recall": 0.0,
            "f1": 0.0,
            "accuracy": 0.0,
        }

    results = metric.compute(predictions=true_predictions, references=true_labels)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

**training the model **

In [52]:
id2label = {i:label for i,label in enumerate(label_names)}
label2id = {label:id for i,label in enumerate(label_names)}

In [53]:
label_names = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

id2label = {i: label for i, label in enumerate(label_names)}
label2id = {label: i for i, label in enumerate(label_names)}

print(f"id2label type: {type(id2label)}")
print(f"label2id type: {type(label2id)}")
print(f"id2label: {id2label}")
print(f"label2id: {label2id}")
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
    num_labels=len(label_names)
)

id2label type: <class 'dict'>
label2id type: <class 'dict'>
id2label: {0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC'}
label2id: {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
model.config.num_labels

7

In [55]:
from transformers import TrainingArguments

args = TrainingArguments(
                         output_dir="wikiann",
                         eval_strategy = "epoch",
                         save_strategy = "epoch",
                         learning_rate = 2e-5,
                         num_train_epochs =3,
                         weight_decay = 0.01)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [56]:
from transformers import Trainer
trainer = Trainer(model = model,
                  args = args,
                  train_dataset = tokenized_datasets['train'],
                  eval_dataset = tokenized_datasets['validation'],
                  data_collator = data_collator,
                  compute_metrics = compute_metrics,
                  tokenizer = tokenizer)
trainer.train()

/tmp/ipython-input-1934604004.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model = model,


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.319200,0.292502,0.747381,0.791814,0.768956,0.908732
2,0.230300,0.288644,0.778145,0.816485,0.796854,0.916683
3,0.168400,0.311123,0.787014,0.822565,0.804397,0.918761


TrainOutput(global_step=7500, training_loss=0.2633173746744792, metrics={'train_runtime': 333.3415, 'train_samples_per_second': 179.996, 'train_steps_per_second': 22.499, 'total_flos': 395751351832320.0, 'train_loss': 0.2633173746744792, 'epoch': 3.0})